In [ ]:
#Imports
import pandas as pd
import math
import collections
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
def load_and_preprocess(standard_scaling=False):
    features = pd.read_csv('data.csv', index_col=0)
    labels = pd.read_csv('labels.csv')

    checkTrue = features.var() == 0
    constantFeatures = [i for i, val in enumerate(checkTrue) if val]
    features = features.drop(features.columns[[constantFeatures]], axis=1)

    nFeatures = np.sort(features.var()).shape[0] #number of features
    features=(features-features.min())/(features.max()-features.min()) #normalize data
    threshold = np.sort(features.var())[nFeatures-5000] #set threshold for variance
    checkTrue = features.var() < threshold
    toRemove = [i for i, val in enumerate(checkTrue) if val]
    features = features.drop(features.columns[[toRemove]], axis=1)

    if standard_scaling:
        scaler = preprocessing.StandardScaler().fit(features)
        features = scaler.transform(features)
        
    pca = PCA(n_components=801)
    pca.fit(features)
    expl = pca.explained_variance_ratio_

    variance = np.cumsum(np.round(expl, decimals=3)*100)
    nComponents = np.where(variance >= 80)[0][0]
    print(f"Number of components that represent 80% of variance:{nComponents}")

    reducedPCA = PCA(n_components=nComponents)
    components = reducedPCA.fit_transform(features)


    return components
    ### 

    

In [ ]:
def createSubsamples(data, M, K, frac=0.8):
    indecies = list(range(data.shape[0]))

    subSamples = []
    for i in range(M):
        subSamples.append(np.random.choice(indecies, size=int(len(indecies)*frac), replace=False))
    return np.array(subSamples)

def createConsensusMatrix(data,subSamples,K, clustering_alg='kmeans'):
    connectivityMatrix = np.zeros(shape=(data.shape[0], data.shape[0]))
    indicatorMatrix = np.zeros(shape=(data.shape[0], data.shape[0]))
    for subSample in subSamples:
        if(clustering_alg == 'kmeans'):
            predictions = (KMeans(n_clusters=K).fit_predict(data[subSample]))
        elif(clustering_alg == 'spectral'):
            predictions = (SpectralClustering(n_clusters=K, n_init=100, assign_labels = 'discretize').fit_predict(data[subSample]))
        for i in range(len(predictions)):
            for j in range(len(predictions)): #set diagonal to zero?
                if predictions[i] == predictions[j]:
                    connectivityMatrix[subSample[i],subSample[j]] += 1
    
    
        for n in range(subSample.shape[0]):
            for m in range(subSample.shape[0]):
                i = subSample[n]
                j = subSample[m]
                indicatorMatrix[i,j] += 1
    return connectivityMatrix/indicatorMatrix


In [ ]:
def PAC_score(nClusters = [3,4,5,6,7], alg='kmeans'):
    legends = [f"clusters = {c}" for c in nClusters]
    pac_scores = []

    for k in nClusters:
        C = createConsensusMatrix(components,subSamples,k, clustering_alg=alg)
        flatten_C = C.flatten()
        ecdf = ECDF(flatten_C)
        x = np.round(ecdf.x, 2)
        u1 = np.where(x >= 0.02)[0][0]
        u2 = np.where(x >= 0.98)[0][0]
        pac_score = ecdf.y[u2]-ecdf.y[u1]
        pac_scores.append(pac_score)
        print(pac_score)
    return pac_scores


In [ ]:
components = load_and_preprocess(standard_scaling=True)
subSamples = createSubsamples(components, 100, 1)
scores = PAC_score(alg='kmeans')


NameError: name 'load_and_preprocess' is not defined

In [ ]:
components = load_and_preprocess(standard_scaling=False)
subSamples = createSubsamples(components, 100, 1)
scores = PAC_score(alg='spectral')

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexes/base.py:4308: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexes/base.py:4308: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
Number of components that represent 80% of variance:135


KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f98e27f7-6444-4cf3-8fec-73bc47540f9b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>